In [143]:
import os
import pandas as pd
from PIL import Image
import torch.nn as nn
import torch
from torch.autograd import Variable
import numpy as np
from torchvision.utils import make_grid


In [169]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torch import autograd
from torch.autograd import Variable
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

In [170]:
pictures_df = pd.read_csv('/Users/test/Desktop/Master_Thesis_Flow/real-nvp/data/GPR_Data/B200/labels_complete_dataset_607.csv')

In [171]:
filename = pictures_df['filename'].values

className = pictures_df['class'].values

file_class_mapping_dict = {}
for i in range(len(filename)):
    file_class_mapping_dict[filename[i]] = className[i]
    


In [172]:
list(pictures_df)

['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']

In [210]:
DataPath = '/Users/test/Desktop/Master_Thesis_Flow/real-nvp/data/GPR_Data/B200/B200_Pictures_GR'
    
dirs = os.listdir(DataPath)


labels = []

images = []

imageName = []
#x= 0
def crop():
    for item in dirs:
        try:
            fullpath = os.path.join(DataPath,item)         #corrected
            if os.path.isfile(fullpath):
                im = Image.open(fullpath).convert('L')
                f, e = os.path.splitext(fullpath)
                imCrop = im.crop((0, 0, 128, 128)) #corrected                
                labels.append(file_class_mapping_dict[item])
                images.append(imCrop)
                imageName.append(item)
        except:
            k =1
            
            
crop()

In [190]:
len(labels)

568

In [211]:
labels[0]

'schichtabloesung'

In [284]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.label_emb = nn.Embedding(10, 10)
        
        self.model = nn.Sequential(
            nn.Linear(128, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.Sigmoid()
        )
    
    def forward(self, x, labels):
       
        x = x.view(1, 128 * 128)
        c = self.label_emb(labels.long())
        x = torch.cat([x, c], 1)
        out = self.model(x)
        return out.squeeze()

In [285]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.label_emb = nn.Embedding(10, 10)
        
        self.model = nn.Sequential(
            nn.Linear(128, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 128),
            nn.Tanh()
        )
    
    def forward(self, z, labels):
        z = z.view(z.size(0), 100)
        c = self.label_emb(labels.long())
        x = torch.cat([z, c], 1)
        out = self.model(x)
       
        return out.view(1, 9, 784)

In [286]:
generator = Generator()
discriminator = Discriminator()

In [287]:
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=1e-4)
g_optimizer = torch.optim.Adam(generator.parameters(), lr=1e-4)

In [288]:
def generator_train_step(batch_size, discriminator, generator, g_optimizer, criterion):
    g_optimizer.zero_grad()
    z = Variable(torch.randn(batch_size, 100)).cuda()
    fake_labels = Variable(torch.LongTensor(np.random.randint(0, 10, batch_size))).cuda()
    fake_images = generator(z, fake_labels)
    validity = discriminator(fake_images, fake_labels)
    g_loss = criterion(validity, Variable(torch.ones(batch_size)).cuda())
    g_loss.backward()
    g_optimizer.step()
    return g_loss.data[0]

In [289]:
def discriminator_train_step(batch_size, discriminator, generator, d_optimizer, criterion, real_images, labels):
    d_optimizer.zero_grad()

    # train with real images
    real_validity = discriminator(real_images, labels)
    print('real size is, ', real_validity.size())
    real_loss = criterion(real_validity, Variable(torch.ones(batch_size)))
    
    # train with fake images
    z = Variable(torch.randn(batch_size, 100)).cuda()
    fake_labels = Variable(torch.LongTensor(np.random.randint(0, 10, batch_size)))
    fake_images = generator(z, fake_labels)
    fake_validity = discriminator(fake_images, fake_labels)
    fake_loss = criterion(fake_validity, Variable(torch.zeros(batch_size)))
    
    d_loss = real_loss + fake_loss
    d_loss.backward()
    d_optimizer.step()
    return d_loss.data[0]

In [290]:
#labels

In [291]:
UniqueLabels = np.unique(labels)

labelsDict = {}

for i, label in enumerate(UniqueLabels):
    labelsDict[label] = i
    
file_label_map_dict = {}
for i in range(len(images)): 
#     print(labels[i])
    file_label_map_dict[filename[i]] = labelsDict[labels[i]]


In [292]:
labels[0]

'schichtabloesung'

In [295]:
num_epochs = 30
n_critic = 5
display_step = 300
g_loss = 0
d_loss = 0
for epoch in range(num_epochs):
    print('Starting epoch {}...'.format(epoch))
    for i, (image) in enumerate(images):
        print(file_label_map_dict[imageName[i]])
#         label = torch.Tensor(Variable(torch.Tensor(file_label_map_dict[imageName[i]])))
        label = torch.Tensor(1)
       
        real_images = Variable(torch.Tensor(np.array(image)).float())
        
        generator.train()
        batch_size = real_images.size(0)
        d_loss = discriminator_train_step(len(real_images), discriminator, generator, d_optimizer, 
                                          criterion, real_images, label)
        

        g_loss = generator_train_step(batch_size, discriminator, generator, g_optimizer, criterion)

#     generator.eval()
    print('g_loss: {}, d_loss: {}'.format(g_loss, d_loss))
#     z = Variable(torch.randn(9, 100))
#     labels = Variable(torch.LongTensor(np.arange(9)))
#     sample_images = generator(z, labels).unsqueeze(1).data.cpu()
#     grid = make_grid(sample_images, nrow=3, normalize=True).permute(1,2,0).numpy()
#     plt.imshow(grid)
#     plt.show()

Starting epoch 0...
4


RuntimeError: size mismatch, m1: [1 x 16394], m2: [128 x 1024] at /Users/soumith/miniconda2/conda-bld/pytorch_1532624435833/work/aten/src/TH/generic/THTensorMath.cpp:2070